In [72]:
import pandas as pd

In [73]:
df = pd.read_csv('jeju.csv',encoding='cp949')
df.head(1)

지역   읍면동          전화번호                           주소  \
0  제주시  일도1동  064-728-4412  제주특별자치도 제주시 중앙로7길 15 (일도일동)   

                             일반현황               청사현황          자치센터현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반  지상 3층  / 연면적 585㎡  개소일 2000-12-20   

      데이터기준일자  
0  2019-06-20

In [74]:
# 1. 주소 정보를 이용해서 위도,경도 값을 저장합니다. '위치' 컬럼에 (위도,경도) 저장
# 주소를 찾아서 저장하는 함수를 작성하여 처리  
# 함수 이름은 geo_coder(query) 리턴값 : (위도,경도)  만일 못찾았다면 '결과없음' 으로 리턴
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [75]:
df['주소'] = df.주소.str.split('(').str[0]

In [76]:
import json

with open('key.json','r') as f:
    key = json.load(f)

In [77]:
def geocoding_1(address,key):
    import requests
    REST_API_KEY = key
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?category_group_code=PO3&query={address}"
    headers = {'Authorization':f'KakaoAK {REST_API_KEY}'}
    res = requests.get(url,headers=headers)
    jsondata = res.json()
    try:
        return jsondata['documents'][0]['y'],jsondata['documents'][0]['x']
    except:
        return '결과없음'
    
####### 도로명주소 위도 경도 값으로 바꿔주기 ########

from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding_2(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude] #위도,경도
        return x_y

    except:
        return '결과없음' #[0,0]

In [101]:
%%time
# df['위치'] = df['주소'].apply(geocoding_1,key=key['key']) #1,20,28,33  ,Wall time: 7.54 s
df['위치'] = df['주소'].apply(geocoding_2) #시간 및 결과없음이 경우에 따라 달라진다.

CPU times: user 952 ms, sys: 127 ms, total: 1.08 s
Wall time: 53.2 s


In [102]:
df.head(5)

지역   읍면동          전화번호                      주소  \
0  제주시  일도1동  064-728-4412   제주특별자치도 제주시 중앙로7길 15    
1  제주시  일도2동  064-728-4442     제주특별자치도 제주시 고마로 28    
2  제주시  이도1동  064-728-1530  제주특별자치도 제주시 중앙로25길 17    
3  제주시  이도2동  064-755-2021     제주특별자치도 제주시 오복3길 8    
4  제주시  삼도1동  064-728-4532     제주특별자치도 제주시 전농로 40    

                             일반현황                        청사현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반           지상 3층  / 연면적 585㎡   
1   인구 38400명 / 면적2.18㎢ / 48통326반   지상 2층·지하 1층 / 연면적 1307.01   
2  인구 7418명 / 면적 0.79㎢ / 16통 97개반           지상 2층 / 2층면적 205㎡   
3  인구 41428명 / 면적 5.4㎢ / 49통 323반  지하 1층·지상 3층 / 연면적 1223.52㎡   
4   인구 14281명 / 면적0.87㎢ / 18통122반   지하 1층·지상3층 / 연면적 2070.37㎡   

                         자치센터현황     데이터기준일자                         위치  
0                개소일 2000-12-20  2019-06-20                       결과없음  
1  총면적 617.22㎡ / 개소일 2000-12-24  2019-06-20  [33.5116166, 126.5382896]  
2                개소일 2000-12-28  2019-06-20   [33.506908, 126.5269967]  
3  총면적 388.9㎡  / 개소일 2000-10-05  2019-06-20    [33.4971019, 126.53526]  
4                   총면적 456.55㎡  2019-06-20  [33.5041806, 126.5174014]

In [103]:
df[df['위치']=='결과없음']

지역   읍면동          전화번호                          주소  \
0    제주시  일도1동  064-728-4412       제주특별자치도 제주시 중앙로7길 15    
13   제주시   오라동  064-728-1541      제주특별자치도 제주시 오라로 10길 6    
20   제주시   애월읍  064-728-8809  제주특별자치도 제주시 애월읍 하귀로 21길 30   
23   제주시   한경면  064-728-7913      제주특별자치도 제주시 한경면 칠전로 37   
32  서귀포시   정방동  064-760-4541     제주특별자치도 서귀포시 태평로421번길 7   

                                                 일반현황  \
0                      인구 3146명 / 면적 0.33㎢ / 11통 61개반   
13                      인구 13729명 / 면적28.7㎢ / 17통74개반   
20           인구 26539명 / 면적202.16㎢ / 26개리70개자연마을317개반   
23  인구 8745명('16.12.31.) / 면적79.123㎢ / 15개리 51개자연마...   
32                                           인구 2444명   

                         청사현황                         자치센터현황     데이터기준일자    위치  
0           지상 3층  / 연면적 585㎡                 개소일 2000-12-20  2019-06-20  결과없음  
13     지하 1층·지상3층 / 연면적815.8㎡    총면적 699.6㎡ / 개소일 2000-12-30  2019-06-20  결과없음  
20    지하 1층·지상2층 / 연면적 618.6㎡    대지면적 1592㎡ / 개소일 2006-03-11  2019-06-20  결과없음  
23         지상 2층 / 연면적 602.4㎡  총면적 3369.71㎡ / 개소일 2002-01-08  2019-06-20  결과없음  
32  지상 3층 지하 1층 / 연면적 769.87㎡                            NaN  2019-06-20  결과없음

In [104]:
# 2. '위치' 컬럼의 값을 Marker 위치값으로, tooltip으로 읍면동을 사용하여 지도에 표시(클리스터이용)
# 함수사용, apply()를 이용해 데이터 프레임에 적용

In [105]:
import folium
from folium.plugins import MarkerCluster

In [106]:
def display(loc,map):
    # print(loc[0],loc[1])
    if loc[0] != '결과없음':
        folium.Marker(loc[0],tooltip=loc[1]).add_to(map)

In [107]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
df[['위치','읍면동']].apply(display,map=map_c,axis=1)
map

In [108]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
map_c = MarkerCluster().add_to(map)
for index, row in df.iterrows():
    # print(index)
    # print(row['읍면동'],row['위치'])
    if row['위치'] != '결과없음':
        folium.Marker(row['위치'],tooltip=row['읍면동']).add_to(map_c)
map

In [109]:
# 3. '일반현황'컬럼의 인구수만 분리하여 '인구수'라는 int형 컬럼을 생성

In [110]:
df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.split().str.get(-1).str.replace('명','').astype(int)

0      3146
1     38400
2      7418
3     41428
4     14281
5      9431
6      8368
7     16701
8     10850
9     25559
10    25000
11     3046
12    28741
13    13729
14    39897
15    56223
16    17606
17     4104
18     2837
19    19925
20    26539
21    15080
22    20804
23     8745
24     2820
25     1718
26    16610
27    19151
28    13866
29    11867
30    11167
31     5207
32     2444
33     4003
34     3798
35     5475
36     4980
37    22225
38    10697
39    10344
40    11310
41    10684
42     3813
Name: 일반현황, dtype: int64

In [111]:
import re

df['인구수'] = df['일반현황'].apply(lambda x:int(re.findall('[0-9]{4,}',x)[-1]))

In [112]:
df.head(2)

지역   읍면동          전화번호                     주소  \
0  제주시  일도1동  064-728-4412  제주특별자치도 제주시 중앙로7길 15    
1  제주시  일도2동  064-728-4442    제주특별자치도 제주시 고마로 28    

                             일반현황                       청사현황  \
0  인구 3146명 / 면적 0.33㎢ / 11통 61개반          지상 3층  / 연면적 585㎡   
1   인구 38400명 / 면적2.18㎢ / 48통326반  지상 2층·지하 1층 / 연면적 1307.01   

                         자치센터현황     데이터기준일자                         위치    인구수  
0                개소일 2000-12-20  2019-06-20                       결과없음   3146  
1  총면적 617.22㎡ / 개소일 2000-12-24  2019-06-20  [33.5116166, 126.5382896]  38400

In [113]:
# 4. 지역을 제주시는 "Jeju", 서귀포시는 "Seogwipo" 변경
df['키'] = df['지역'].replace(['제주시','서귀포시'],["Jeju","Seogwipo"]) 

In [114]:
df.sample(3)

지역   읍면동          전화번호                         주소  \
6    제주시  용담1동  064-728-4592       제주특별자치도 제주시 용남3길 11    
26  서귀포시   대정읍  064-760-4012  제주특별자치도 서귀포시 대정읍 하모중앙로 20   
13   제주시   오라동  064-728-1541     제주특별자치도 제주시 오라로 10길 6    

                                  일반현황                              청사현황  \
6         인구 8368명 / 면적0.61㎢ / 15통102반  지상 3층 / 대지면적 8825㎡ / 건축면적 428.55   
26  인구 16610명 / 면적 78.53㎢ / 23개리 240개반     지상 2층·지하 1층 / 연면적 986.95㎡(본관)   
13       인구 13729명 / 면적28.7㎢ / 17통74개반            지하 1층·지상3층 / 연면적815.8㎡   

                           자치센터현황     데이터기준일자                         위치  \
6   연면적 112049㎡  / 개소일 2000-12-24  2019-06-20  [33.5091076, 126.5133723]   
26      총면적 149㎡ / 개소일 2006-08-29  2019-06-20  [33.2268141, 126.2521371]   
13    총면적 699.6㎡ / 개소일 2000-12-30  2019-06-20                       결과없음   

      인구수         키  
6    8368      Jeju  
26  16610  Seogwipo  
13  13729      Jeju

In [115]:
# 5. 지역별 데이터 집계
df1 = df.groupby('키')['인구수'].sum()
df1= df1.to_frame().reset_index()

In [116]:
df1

키     인구수
0      Jeju  462396
1  Seogwipo  167641

In [117]:
import folium,json

In [118]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
jsonfile = open('jeju-municipalities-geo.json','r')
jsondata = json.load(jsonfile)
# jsondata
folium.Choropleth(geo_data=jsondata,
                  data=df1,
                  columns=['키','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map

In [119]:
# 6. 읍면동별로 인구수를 표시

In [120]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
jsonfile = open('jeju.json','r')
jsondata = json.load(jsonfile)
# jsondata
folium.Choropleth(geo_data=jsondata,
                  data=df,
                  columns=['읍면동','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map

In [121]:
jsonfile = open('HangJeongDong_ver20230101.geojson','r',encoding='utf8').read()
jsondata = json.loads(jsonfile)
# jsondata
jsondata_jeju = {'type': 'FeatureCollection'}
jsondata_pick = []
jsondata_dong = []
for idx in jsondata['features']:
    if idx['properties']['sidonm'] == "제주특별자치도":
        dong = idx['properties']['adm_nm'].split()[-1].strip()
        # print(dong)
        jsondata_dong.append(dong)
        idx['id']=dong
        jsondata_pick.append(idx)
jsondata_jeju['features']=jsondata_pick

In [122]:
# jsondata_jeju

In [123]:
map = folium.Map((33.4996213,126.5311884),zoom_start=8)
# jsondata
folium.Choropleth(geo_data=jsondata_jeju,
                  data=df,
                  columns=['읍면동','인구수'],
                  key_on='feature.id',
                  legend_name='제주 인구수',
                  bins=10,fill_color='YlOrBr').add_to(map)
map